In [1]:
import sys
import math
sys.path.append('/home/afomin/projects/mj/quad/QTL/src/')

In [2]:
from pid import PIDController as pid

In [3]:
kp = 7.5
ki = 0.1
kd = 0.5

dt = 0.001

tar = 0.0

In [4]:
p = pid(kp=kp, ki=ki, kd=kd)

In [5]:
print('working range -30...+30 degr, or -0.5...+0.5 rads')
out = []
inp = []
for i in range(-5, 6, 1):
    inp.append(i/10.0)
print(", ".join(f"{i:.2f}" for i in inp))

for i in inp:
    out.append(p.update(tar, i, dt))
print(", ".join(f"{i:.2f}" for i in out))

working range -30...+30 degr, or -0.5...+0.5 rads
-0.50, -0.40, -0.30, -0.20, -0.10, 0.00, 0.10, 0.20, 0.30, 0.40, 0.50
3.75, 3.00, 2.25, 1.50, 0.75, 0.00, -0.75, -1.50, -2.25, -3.00, -3.75


In [6]:
print('working range -180...+180 degr, or -3.14...+3.14 rads')
out = []
inp = []
for i in range(-180, 180, 20):
    inp.append(math.radians(i))
print(", ".join(f"{i:.2f}" for i in inp))

for i in inp:
    out.append(p.update(tar, i, dt))
print(", ".join(f"{i:.2f}" for i in out))

working range -180...+180 degr, or -3.14...+3.14 rads
-3.14, -2.79, -2.44, -2.09, -1.75, -1.40, -1.05, -0.70, -0.35, 0.00, 0.35, 0.70, 1.05, 1.40, 1.75, 2.09, 2.44, 2.79
23.56, 20.94, 18.33, 15.71, 13.09, 10.47, 7.85, 5.24, 2.62, 0.00, -2.62, -5.24, -7.85, -10.47, -13.09, -15.71, -18.33, -20.94


In [7]:
"""
понять, в каком диапазоне вернут результат эти функции
self.angles['roll'] = math.atan2(acc_y, acc_z)
self.angles['pitch'] = math.atan2(-acc_x, acc_z)
возвращает в диапазоне минус пи и плюс пи
"""
"""
ПИД регулятор выдает проценты мощности мотора.
Плюс надо учитывать, что у меня 3 ПИД регулятора, которые могут давать суммарный вклад
Итого может быть 3X от максимально выдаваемых значений ПИД регулятора


Какая логика - есть троттл, который я подаю, он может быть от 0 до 100%
Т.е. это то значение, которое приходит от пульта

Мы берем этот троттл, применяем наши ПИД значения, может получиться что-то вроде

-3xPID.... 100% + 3xPID

Мы берем эти значения и делаем так, чтобы минимальное значение стало  - т.е. 

Если на пульте 0 - что мы хотим? Мы хотим выключить моторы, ПИДы не работают,
ошибки с них игнорируются и ничего не накапливается в них. Нужно сделать дернуть reset  метод в них?

Если на пульте > 0 - мы хотим чтобы ПИДЫ работали как и должны

Пиды могут дать значение < 0, а также значение > 100. Нас это не устраивает.
Это должно обрабатываться после ПИДов. Как?
Во-первых у нас не должно быть ситуации, когда на одном моторе < 0, на другом > 0.
Это будет исключено выборов коэффициентов ПИД регулятора.

Логика,
1) если у нас < 0, мы берем все значения и сдвигаем их до +10 (MIN_ACTIVE_THROTTLE) вверх
2) Дальше мы проверяем, что у нас > 100% (например 110%). Если что-то есть, то мы берем и считаем (110 - 100 = 10)
Эти 10% мы вычитаем из каждого значения моторов.
3) При этом мы финально проверяем значение моторов. Они не могут быть меньше, чем MIN_ACTIVE_THROTTLE

С такой логикой:
1) мы не можем получить ситуацию, когда у нас < MIN_ACTIVE_THROTTLE мощность
2) мы не можем получить ситуацию, когда у нас > 100% мощность подается на моторы



мы хотим минимальное значение на моторах.
Если мы хотим выключить моторы, мы делаем полный 

0...

"""



'\nПИД регулятор выдает проценты мощности мотора.\nПлюс надо учитывать, что у меня 3 ПИД регулятора, которые могут давать суммарный вклад\nИтого может быть 3X от максимально выдаваемых значений ПИД регулятора\n'